# Convnets in TensorFlow

## Agenda
* Plaing with convolutions
* Convolution support in TF
* More MNIST
* Autoencorder

### convolutionを理解する
#### 数学 or 物理における Convolution
integrationによって与えられた2つの関数から派生した関数。
それは片方がもう一方によって修正されたshapeがどのようなものかを表現している
#### NNにおけるConvolution
*要素幅の乗算* によって与えられた2つの関数から派生した関数。
それは片方がもう一方によって修正された *shapeと値* がどのようなものかを表現している

1つのconvolution layerを使うことで画像を修正できる

```
tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
```